# Physics Informed FNO for Magnetohydrodynamics Equations

In this notebook, we will study the application of physics informed data-driven modeling to the incompressible magnetohydrodynamics (MHD) equations representing an incompressible fluid in the presence of a magnetic field $\mathbf{B}$. Our model will be built using a Tensor Factorized Fourier Neural Operator (tFNO), and trained in conjunction with the PDEs representing our system. The model is physics-informed during training by encoding known information about the physical system into the loss functions,  enabling generalization of the resulting model to a variety of settings in the solution space. Specifically, the PDEs and initial conditions are used as soft constraints learned by the neural network as its trains. Models covering different data regimes governed by the Reynolds number are trained using transfer learning to showcase how our model may be applied to both laminar and turbulent flows. The AI-accelerated surrogate model is compared to classical simulations to compare its throughput and accuracy.

Note that while the majority of the code needed to run this example is provided in the notebook, the lower barrier to entry for training and evaluating models will be to run the scripts in the source directory, and the material referenced here should be used as a base for learning the underlying components leading to model training and evaluation. 

#### Learning Outcomes
* How to apply physics constraints to neural networks
* Learn how the Tensor Factorized Fourier Neural Operator can be applied to physics based problems
* Learn how to define PDEs with PhysicsNeMo
* Train PINOs with PhysicsNeMo Core
* Learn how data driven modeling can help build computationally efficient surrogates for physics problems

## Problem Overview

To examine the properties of PINOs with multiple complex equations, we examined the ability of the networks to reproduce the incompressible magnetohydrodynamics (MHD) equations representing an incompressible fluid in the presence of a magnetic field $\mathbf{B}$. These equations are present in several astrophysical phenomena, including black hole accretion and binary neutron star mergers. Additionally, MDH has applications to nuclear power engineering, and plasma modeling. 

These equations for incompressible MHD are given by:

$$\begin{align*}
\partial_t \mathbf{u}+\mathbf{u} \cdot \nabla \mathbf{u} &=
-\nabla \left( p+\frac{B^2}{2} \right)/\rho_0 +\mathbf{B}
\cdot \nabla \mathbf{B}+\nu \nabla^2 \mathbf{u}, \\
\partial_t \mathbf{B}+\mathbf{u} \cdot \nabla \mathbf{B} &=
\mathbf{B} \cdot \nabla \mathbf{u}+\eta \nabla^2 \mathbf{B}, \\
\nabla \cdot \mathbf{u} &= 0, \\
\nabla \cdot \mathbf{B} &= 0,
\end{align*}$$
 
where $\mathbf{u}$ is the velocity field, $p$ is  the pressure, $B^2$ is the magnitude of the magnetic field, $\rho_0=1$  is the density of the fluid, $\nu$ is the kinetic viscosity,  and $\eta$ is the magnetic resistivity.  We have two equations for evolution and two constraint equations.


For the magnetic field divergence equation, we can either include it in the loss function or instead evolve the magnetic vector potential $\mathbf{A}$. This quantity is defined such that

$$\begin{align*}
\mathbf{B} = \nabla \times \mathbf{A},
\end{align*}$$

which ensures that the divergence of $\mathbf{B}$ is zero. By evolving magnetic vector potential $\mathbf{A}$ instead of the magnetic field $\mathbf{B}$, we have a new evolution equation for the vector potential $\mathbf{A}$. This equation is given by 

$$\begin{align*}
\partial_t \mathbf{A} + \mathbf{u} \cdot \nabla \mathbf{A}=\eta \nabla^2 \mathbf{A}.
\end{align*}$$

In practice, using the magnetic vector potential representation leads to better model performance. 

## Data Creation
To train our model, a representative dataset is first created that gives enough coverage of the solution space to train a surrogate model to make predictions on new data points. To obtain interesting results without additional computational difficulty, we will solve the equations in 2D with periodic boundary conditions. This results in solving a total of 3 evolution PDEs at each time step. Two for the velocity evolution, and one for the magnetic vector potential. 

The solution space to this problem can be obtained numerically by solving the PDEs from above with a numerical solver such as `dedalus`. To generate this data, `dedalus` is used to simulate a 2D periodic incompressible MHD flow with a passive tracer field for visualization. The initial flow is in the $x$-direction and depends only on $z$. The problem is non-dimensionalized using the shear-layer spacing and velocity jump, so the resulting viscosity and tracer diffusivity are related to the Reynolds and
Schmidt numbers as:

$$\begin{align}
\nu &= \frac{1}{\text{Re}} \\
\eta &= \frac{1}{\text{Re}_M} \\
D &= \frac{\nu}{\text{Sc}}
\end{align}$$

The initial data field for running the simulation is produced using the Gaussian Random Field method in which the radial basis function kernel (RBF) is transformed into Fourier space to obey the desired periodic boundary conditions. Finally, two initial data fields the vorticity potential and magnetic potential are used to guarantee initial velocity and magnetic fields are divergence free. 

The dataset is produced by running 1,000 simulations with different initial conditions, and evolving the system for 1,000 time steps. The time step used is $\Delta t=0.001s$, however output data is saved at an interval of $t=0.01$ for a total time of 1second, resulting in 101 samples per simulation. 

## Defining our Constraints - Setting up the PDE

Constraints are used to define the objectives for training our model. They house a set of nodes from which a computational graph is build for execution as well as the loss function. [PhysicsNeMo Sim](https://docs.nvidia.com/deeplearning/physicsnemo/physicsnemo-sym/index.html) provides utilities tailored for physics-informed machine learning, and uses abstracted APIs that allow users to think and model the problem from the lens of equations, constraints, etc. In this example, we will only leverage the physics-informed utilities to see how we can add physics to an existing data-driven model with ease while still maintaining the flexibility to define our own training loop and other details. The types of constraints used will be problem dependent. For this example, we can define the following constraints: 

**Data Loss**: Obtain simulation data and compare it to the PINO output.

**PDE Loss**: Use the known PDEs of the system to describe violations of the time evolution of our system

**Constraint Loss**: This loss describes constraints from the PDE. Specifically, the velocity divergence free condition of equation 3 and magnetic divergence free condition of equation 4.

**Initial Condition Loss**: Input field compared to the output at $t=0$

**Boundary Condition Loss**: Difference in boundary terms. In our case, we have a periodic boundary constraint.



To begin setting up our constraints, we can start by defining the MHD equations using the `PDE` class from `physicsnemo.sym.eq.pde`. The process of converting our PDEs into a form that is compatible with `PhysicsNeMo` involves defining a class to hold our equations, called `MHD_PDE`, and including each term of the equations. Each variable of the equations is set up as a `Sympy` `Function`, which is then used to create an attribute of our `MHD_PDE` class that holds the final `equations`.

Because we have elected to solve the equations in two dimensions, we only have the input variables $x$, $y$, $t$ and and the Laplacian operator. 

In PhysicsNeMo, it is preferable to represent our equations by isolating our target terms on the left, and moving the rest of the equation to the right-hand-side. To do this, various components of each equation are compartmentalized, and the final set of equations is composed from these parts.

```python
from physicsnemo.sym.eq.pde import PDE
from sympy import Symbol, Function, Number


class MHD_PDE(PDE):
    """MHD PDEs using PhysicsNeMo Sym"""

    name = "MHD_PDE"

    def __init__(self, nu=1e-4, eta=1e-4, rho0=1.0):

        # x, y, time
        x, y, t, lap = Symbol("x"), Symbol("y"), Symbol("t"), Symbol("lap")

        # make input variables
        input_variables = {"x": x, "y": y, "t": t, "lap": lap}

        # make functions
        u = Function("u")(*input_variables)
        v = Function("v")(*input_variables)
        Bx = Function("Bx")(*input_variables)
        By = Function("By")(*input_variables)
        A = Function("A")(*input_variables)
        # pressure
        ptot = Function("ptot")(*input_variables)

        u_rhs = Function("u_rhs")(*input_variables)
        v_rhs = Function("v_rhs")(*input_variables)
        Bx_rhs = Function("Bx_rhs")(*input_variables)
        By_rhs = Function("By_rhs")(*input_variables)
        A_rhs = Function("A_rhs")(*input_variables)

        # initialize constants
        nu = Number(nu)
        eta = Number(eta)
        rho0 = Number(rho0)

        # set equations
        self.equations = {}

        # u · ∇u
        self.equations["vel_grad_u"] = u * u.diff(x) + v * u.diff(y)
        self.equations["vel_grad_v"] = u * v.diff(x) + v * v.diff(y)
        # B · ∇u
        self.equations["B_grad_u"] = Bx * u.diff(x) + v * Bx.diff(y)
        self.equations["B_grad_v"] = Bx * v.diff(x) + By * v.diff(y)
        # u · ∇B
        self.equations["vel_grad_Bx"] = u * Bx.diff(x) + v * Bx.diff(y)
        self.equations["vel_grad_By"] = u * By.diff(x) + v * By.diff(y)
        # B · ∇B
        self.equations["B_grad_Bx"] = Bx * Bx.diff(x) + By * Bx.diff(y)
        self.equations["B_grad_By"] = Bx * By.diff(x) + By * By.diff(y)
        # ∇ × (u × B) = u(∇ · B) - B(∇ · u) B · ∇u − u · ∇B
        self.equations["uBy_x"] = u * By.diff(x) + By * u.diff(x)
        self.equations["uBy_y"] = u * By.diff(y) + By * u.diff(y)
        self.equations["vBx_x"] = v * Bx.diff(x) + Bx * v.diff(x)
        self.equations["vBx_y"] = v * Bx.diff(y) + Bx * v.diff(y)
        # ∇ · B 
        self.equations["div_B"] = Bx.diff(x) + By.diff(y)
        # ∇ · u 
        self.equations["div_vel"] = u.diff(x) + v.diff(y)

        # RHS of MHD equations

        self.equations["u_rhs"] = (
            -self.equations["vel_grad_u"]
            - ptot.diff(x) / rho0
            + self.equations["B_grad_Bx"] / rho0
            + nu * u.diff(lap)
        )
        self.equations["v_rhs"] = (
            -self.equations["vel_grad_v"]
            - ptot.diff(y) / rho0
            + self.equations["B_grad_By"] / rho0
            + nu * v.diff(lap)
        )
        u * By.diff(y) + By * u.diff(y) - v * Bx.diff(y) + Bx * v.diff(y)
        self.equations["Bx_rhs"] = (
            self.equations["uBy_y"] - self.equations["vBx_y"] + eta * Bx.diff(lap)
        )
        self.equations["By_rhs"] = -(
            self.equations["uBy_x"] - self.equations["vBx_x"]
        ) + eta * By.diff(lap)

        # Final equations move all terms to RHS
        self.equations["Du"] = u.diff(t) - u_rhs
        self.equations["Dv"] = v.diff(t) - v_rhs
        self.equations["DBx"] = Bx.diff(t) - Bx_rhs
        self.equations["DBy"] = By.diff(t) - By_rhs

        # Vec potential equations
        self.equations["vel_grad_A"] = u * A.diff(x) + v * A.diff(y)
        self.equations["A_rhs"] = -self.equations["vel_grad_A"] + eta * A.diff(lap)
        self.equations["DA"] = A.diff(t) - A_rhs
```

Our model's output can then be used to compute the loss between prediction and true values, and for computing loss based on initial conditions, PDEs, and simulation data.


## Defining our Constraints - Loss Functions 

Now that we have defined our PDE, we can define all of the constraints that make up the loss function for our problem. The loss functions are defined inside of a class called `LossMHD_PhysicsNeMo`, which can use a weighted sum of individual losses for training. Additionally, all of the fixed and constant parameters needed are added to the class definition.

```python
import torch
import torch.nn.functional as F
from physicsnemo.models.layers.spectral_layers import fourier_derivatives

from .losses import (LpLoss, fourier_derivatives_lap, fourier_derivatives_ptot,
                     fourier_derivatives_vec_pot)
from .mhd_pde import MHD_PDE


class LossMHDVecPot_PhysicsNeMo(object):
    "Calculate loss for MHD equations with vector potential, using physicsnemo derivatives"

    def __init__(
        self,
        nu=1e-4,
        eta=1e-4,
        rho0=1.0,
        data_weight=1.0,
        ic_weight=1.0,
        pde_weight=1.0,
        constraint_weight=1.0,
        use_data_loss=True,
        use_ic_loss=True,
        use_pde_loss=True,
        use_constraint_loss=True,
        u_weight=1.0,
        v_weight=1.0,
        A_weight=1.0,
        Du_weight=1.0,
        Dv_weight=1.0,
        DA_weight=1.0,
        div_B_weight=1.0,
        div_vel_weight=1.0,
        Lx=1.0,
        Ly=1.0,
        tend=1.0,
        use_weighted_mean=False,
        **kwargs,
    ):  # add **kwargs so that we ignore unexpected kwargs when passing a config dict):

        self.nu = nu
        self.eta = eta
        self.rho0 = rho0
        self.data_weight = data_weight
        self.ic_weight = ic_weight
        self.pde_weight = pde_weight
        self.constraint_weight = constraint_weight
        self.use_data_loss = use_data_loss
        self.use_ic_loss = use_ic_loss
        self.use_pde_loss = use_pde_loss
        self.use_constraint_loss = use_constraint_loss
        self.u_weight = u_weight
        self.v_weight = v_weight
        self.Du_weight = Du_weight
        self.Dv_weight = Dv_weight
        self.div_B_weight = div_B_weight
        self.div_vel_weight = div_vel_weight
        self.Lx = Lx
        self.Ly = Ly
        self.tend = tend
        self.use_weighted_mean = use_weighted_mean
        self.A_weight = A_weight
        self.DA_weight = DA_weight
        # Define 2D MHD PDEs
        self.mhd_pde_eq = MHD_PDE(self.nu, self.eta, self.rho0)
        self.mhd_pde_node = self.mhd_pde_eq.make_nodes()

        if not self.use_data_loss:
            self.data_weight = 0
        if not self.use_ic_loss:
            self.ic_weight = 0
        if not self.use_pde_loss:
            self.pde_weight = 0
        if not self.use_constraint_loss:
            self.constraint_weight = 0

    def __call__(self, pred, true, inputs, return_loss_dict=False):
        loss, loss_dict = self.compute_losses(pred, true, inputs)
        return loss, loss_dict

    def compute_losses(self, pred, true, inputs):
        "Compute weighted loss and dictionary"
        pred = pred.reshape(true.shape)
        u = pred[..., 0]
        v = pred[..., 1]
        A = pred[..., 2]

        loss_dict = {}

        # Data
        if self.use_data_loss:
            loss_data, loss_u, loss_v, loss_A = self.data_loss(
                pred, true, return_all_losses=True
            )
            loss_dict["loss_data"] = loss_data
            loss_dict["loss_u"] = loss_u
            loss_dict["loss_v"] = loss_v
            loss_dict["loss_A"] = loss_A
        else:
            loss_data = 0
        # IC
        if self.use_ic_loss:
            loss_ic, loss_u_ic, loss_v_ic, loss_A_ic = self.ic_loss(
                pred, inputs, return_all_losses=True
            )
            loss_dict["loss_ic"] = loss_ic
            loss_dict["loss_u_ic"] = loss_u_ic
            loss_dict["loss_v_ic"] = loss_v_ic
            loss_dict["loss_A_ic"] = loss_A_ic
        else:
            loss_ic = 0

        # PDE
        if self.use_pde_loss:
            Du, Dv, DA = self.mhd_pde(u, v, A)
            loss_pde, loss_Du, loss_Dv, loss_DA = self.mhd_pde_loss(
                Du, Dv, DA, return_all_losses=True
            )
            loss_dict["loss_pde"] = loss_pde
            loss_dict["loss_Du"] = loss_Du
            loss_dict["loss_Dv"] = loss_Dv
            loss_dict["loss_DA"] = loss_DA
        else:
            loss_pde = 0

        # Constraints
        if self.use_constraint_loss:
            div_vel, div_B = self.mhd_constraint(u, v, A)
            loss_constraint, loss_div_vel, loss_div_B = self.mhd_constraint_loss(
                div_vel, div_B, return_all_losses=True
            )
            loss_dict["loss_constraint"] = loss_constraint
            loss_dict["loss_div_vel"] = loss_div_vel
            loss_dict["loss_div_B"] = loss_div_B
        else:
            loss_constraint = 0

        if self.use_weighted_mean:
            weight_sum = (
                self.data_weight
                + self.ic_weight
                + self.pde_weight
                + self.constraint_weight
            )
        else:
            weight_sum = 1.0

        loss = (
            self.data_weight * loss_data
            + self.ic_weight * loss_ic
            + self.pde_weight * loss_pde
            + self.constraint_weight * loss_constraint
        ) / weight_sum
        loss_dict["loss"] = loss
        return loss, loss_dict

```

The MDH equations that we defined before are initialized for use within the  following loss functions. 

### Data Loss
The data loss is used to compare simulation data to the output of our model. The velocity in $x$ and $y$, as well as magnetic vector potential \mathbf{A} is directly compared to the ground truth data through the `Lp-Loss`, and the relative mean squared error is returned. 


```python
def data_loss(self, pred, true, return_all_losses=False):
    "Compute data loss"
    lploss = LpLoss(size_average=True)
    u_pred = pred[..., 0]
    v_pred = pred[..., 1]
    A_pred = pred[..., 2]

    u_true = true[..., 0]
    v_true = true[..., 1]
    A_true = true[..., 2]

    loss_u = lploss(u_pred, u_true)
    loss_v = lploss(v_pred, v_true)
    loss_A = lploss(A_pred, A_true)

    if self.use_weighted_mean:
        weight_sum = self.u_weight + self.v_weight + self.A_weight
    else:
        weight_sum = 1.0

    loss_data = (
        self.u_weight * loss_u + self.v_weight * loss_v + self.A_weight * loss_A
    ) / weight_sum

    if return_all_losses:
        return loss_data, loss_u, loss_v, loss_A
    else:
        return loss_data
```

## PDE Loss
The PDE loss describes violations of the time evolution of the PDEs and the PINO outputs. In order to make this comparison, the spatial and temporal derivatives of the output fields need to be computed. To do so, Fourier differentiation is used to calculate the spacial derivatives, and second order finite differencing is used for temporal derivatives. The output fields are the velocity in the $x$ direction ($u$), the velocity in the $y$ direction ($v$), and the magnetic vector potential (\mathbf{A}). The PDE loss is then defined as the MSE loss between zero and the PDE, after putting all the terms on the same side of the equation.

```python
def mhd_pde(self, u, v, A, p=None):
    "Compute PDEs for MHD using vector potential"
    nt = u.size(1)
    nx = u.size(2)
    ny = u.size(3)
    dt = self.tend / (nt - 1)

    # compute fourier derivatives
    f_du, _ = fourier_derivatives(u, [self.Lx, self.Ly])
    f_dv, _ = fourier_derivatives(v, [self.Lx, self.Ly])
    f_dBx, f_dBy, f_dA, f_dB, B2_h = fourier_derivatives_vec_pot(
        A, [self.Lx, self.Ly]
    )

    u_x = f_du[:, 0:nt, :nx, :ny]
    u_y = f_du[:, nt : 2 * nt, :nx, :ny]
    v_x = f_dv[:, 0:nt, :nx, :ny]
    v_y = f_dv[:, nt : 2 * nt, :nx, :ny]
    A_x = f_dA[:, 0:nt, :nx, :ny]
    A_y = f_dA[:, nt : 2 * nt, :nx, :ny]

    Bx = f_dB[:, 0:nt, :nx, :ny]
    By = f_dB[:, nt : 2 * nt, :nx, :ny]
    Bx_x = f_dBx[:, 0:nt, :nx, :ny]
    Bx_y = f_dBx[:, nt : 2 * nt, :nx, :ny]
    By_x = f_dBy[:, 0:nt, :nx, :ny]
    By_y = f_dBy[:, nt : 2 * nt, :nx, :ny]

    u_lap = fourier_derivatives_lap(u, [self.Lx, self.Ly])
    v_lap = fourier_derivatives_lap(v, [self.Lx, self.Ly])
    A_lap = fourier_derivatives_lap(A, [self.Lx, self.Ly])

    # note that for pressure, the zero mode (the mean) cannot be zero for invertability so it is set to 1
    div_vel_grad_vel = u_x**2 + 2 * u_y * v_x + v_y**2
    div_B_grad_B = Bx_x**2 + 2 * Bx_y * By_x + By_y**2
    f_dptot = fourier_derivatives_ptot(
        p, div_vel_grad_vel, div_B_grad_B, B2_h, self.rho0, [self.Lx, self.Ly]
    )
    ptot_x = f_dptot[:, 0:nt, :nx, :ny]
    ptot_y = f_dptot[:, nt : 2 * nt, :nx, :ny]

    # Plug inputs into dictionary
    all_inputs = {
        "u": u,
        "u__x": u_x,
        "u__y": u_y,
        "v": v,
        "v__x": v_x,
        "v__y": v_y,
        "Bx": Bx,
        "Bx__x": Bx_x,
        "Bx__y": Bx_y,
        "By": By,
        "By__x": By_x,
        "By__y": By_y,
        "A__x": A_x,
        "A__y": A_y,
        "ptot__x": ptot_x,
        "ptot__y": ptot_y,
        "u__lap": u_lap,
        "v__lap": v_lap,
        "A__lap": A_lap,
    }

    # Substitute values into PDE equations
    u_rhs = self.mhd_pde_node[14].evaluate(all_inputs)["u_rhs"]
    v_rhs = self.mhd_pde_node[15].evaluate(all_inputs)["v_rhs"]
    A_rhs = self.mhd_pde_node[23].evaluate(all_inputs)["A_rhs"]

    u_t = self.Du_t(u, dt)
    v_t = self.Du_t(v, dt)
    A_t = self.Du_t(A, dt)

    # Find difference
    Du = self.mhd_pde_node[18].evaluate({"u__t": u_t, "u_rhs": u_rhs[:, 1:-1]})[
        "Du"
    ]
    Dv = self.mhd_pde_node[19].evaluate({"v__t": v_t, "v_rhs": v_rhs[:, 1:-1]})[
        "Dv"
    ]
    DA = self.mhd_pde_node[24].evaluate({"A__t": A_t, "A_rhs": A_rhs[:, 1:-1]})[
        "DA"
    ]

    return Du, Dv, DA


def mhd_pde_loss(self, Du, Dv, DA, return_all_losses=None):
    "Compute PDE loss"
    Du_val = torch.zeros_like(Du)
    Dv_val = torch.zeros_like(Dv)
    DA_val = torch.zeros_like(DA)

    loss_Du = F.mse_loss(Du, Du_val)
    loss_Dv = F.mse_loss(Dv, Dv_val)
    loss_DA = F.mse_loss(DA, DA_val)

    if self.use_weighted_mean:
        weight_sum = self.Du_weight + self.Dv_weight + self.DA_weight
    else:
        weight_sum = 1.0

    loss_pde = (
        self.Du_weight * loss_Du
        + self.Dv_weight * loss_Dv
        + self.DA_weight * loss_DA
    ) / weight_sum

    if return_all_losses:
        return loss_pde, loss_Du, loss_Dv, loss_DA
    else:
        return loss_pde
```

## Constraint Loss
The constraint illustrates the deviations of the velocity divergence free condition of Eq3 and the magnetic divergence free condition of Eq4. These conditions are implemented similarly to the PDE loss, but without time derivative terms. The constraint loss is then the MSE between each of the constraint equations and zero.  

```python
def mhd_constraint(self, u, v, A):
    "Compute constraints"
    nt = u.size(1)
    nx = u.size(2)
    ny = u.size(3)

    f_du, _ = fourier_derivatives(u, [self.Lx, self.Ly])
    f_dv, _ = fourier_derivatives(v, [self.Lx, self.Ly])
    f_dBx, f_dBy, _, _, _ = fourier_derivatives_vec_pot(A, [self.Lx, self.Ly])

    u_x = f_du[:, 0:nt, :nx, :ny]
    v_y = f_dv[:, nt : 2 * nt, :nx, :ny]
    Bx_x = f_dBx[:, 0:nt, :nx, :ny]
    By_y = f_dBy[:, nt : 2 * nt, :nx, :ny]

    div_B = self.mhd_pde_node[12].evaluate({"Bx__x": Bx_x, "By__y": By_y})["div_B"]
    div_vel = self.mhd_pde_node[13].evaluate({"u__x": u_x, "v__y": v_y})["div_vel"]

    return div_vel, div_B

def mhd_constraint_loss(self, div_vel, div_B, return_all_losses=False):
        "Compute constraint loss"
        div_vel_val = torch.zeros_like(div_vel)
        div_B_val = torch.zeros_like(div_B)

        loss_div_vel = F.mse_loss(div_vel, div_vel_val)
        loss_div_B = F.mse_loss(div_B, div_B_val)

        if self.use_weighted_mean:
            weight_sum = self.div_vel_weight + self.div_B_weight
        else:
            weight_sum = 1.0

        loss_constraint = (
            self.div_vel_weight * loss_div_vel + self.div_B_weight * loss_div_B
        ) / weight_sum

        if return_all_losses:
            return loss_constraint, loss_div_vel, loss_div_B
        else:
            return loss_constraint
```

## Initial Condition Loss
The initial condition loss encourages the model to associate the input field with the output field specifically at t=0. This constraint can usually be achieved with data loss, however this approach emphasized the importance of correct initial condition prediction, and enables training in the absence of data. Training without data and the significance of the initial condition term stem from the PDE loss term. 

```python
def ic_loss(self, pred, input, return_all_losses=False):
    "Compute initial condition loss"
    lploss = LpLoss(size_average=True)
    ic_pred = pred[:, 0]
    ic_true = input[:, 0, ..., 3:]
    u_ic_pred = ic_pred[..., 0]
    v_ic_pred = ic_pred[..., 1]
    A_ic_pred = ic_pred[..., 2]

    u_ic_true = ic_true[..., 0]
    v_ic_true = ic_true[..., 1]
    A_ic_true = ic_true[..., 2]

    loss_u_ic = lploss(u_ic_pred, u_ic_true)
    loss_v_ic = lploss(v_ic_pred, v_ic_true)
    loss_A_ic = lploss(A_ic_pred, A_ic_true)

    if self.use_weighted_mean:
        weight_sum = self.u_weight + self.v_weight + self.A_weight
    else:
        weight_sum = 1.0

    loss_ic = (
        self.u_weight * loss_u_ic
        + self.v_weight * loss_v_ic
        + self.A_weight * loss_A_ic
    ) / weight_sum

    if return_all_losses:
        return loss_ic, loss_u_ic, loss_v_ic, loss_A_ic
    else:
        return loss_ic
```

Similar to the initial condition loss, boundary condition loss can be used to describe violations of the boundary terms. In this specific case, the tFNO architecture ensures that the periodic boundary conditions are satisfied, thus the term is not used in this example. 

In theory, training can be done by correctly predicting the initial conditions, boundary conditions and correctly evolving the PDE forward in time. In practice, having data helps the model converge more quickly. However, an incorrect initial condition results in the PDE evolving the wrong state forward in time, which is why it is emphasized as its own term. The initial condition loss is calculated by taking the input fields and computing the relative MSE with output fields at t=0. 


## Dataset and Dataloaders
To use the data we have generated, we need to define a dataset and a dataloader that can ingest the files and parse them based on the relevant content. 

```python
import glob
import os

import h5py
from torch.utils import data


class Dedalus2DDataset(data.Dataset):
    "Dataset for MHD 2D Dataset"

    def __init__(
        self,
        data_path,
        output_names="output-????",
        field_names=["magnetic field", "velocity"],
        num_train=None,
        num_test=None,
        use_train=True,
    ):
        self.data_path = data_path
        self.output_names = output_names
        raw_path = os.path.join(data_path, output_names, "*.h5")
        files_raw = sorted(glob.glob(raw_path))
        self.files_raw = files_raw
        self.num_files_raw = num_files_raw = len(files_raw)
        self.field_names = field_names
        self.use_train = use_train

        if (num_train is None) or (num_train > num_files_raw):
            num_train = num_files_raw
        self.num_train = num_train
        self.train_files = self.files_raw[:num_train]
        if (num_test is None) or (num_test > (num_files_raw - num_train)):
            num_test = num_files_raw - num_train
        self.num_test = num_test
        self.test_end = test_end = num_train + num_test
        self.test_files = self.files_raw[num_train:test_end]
        if (self.use_train) or (self.test_files is None):
            files = self.train_files
        else:
            files = self.test_files
        self.files = files
        self.num_files = num_files = len(files)

    def __len__(self):
        length = len(self.files)
        return length

    def __getitem__(self, index):
        "Gets item for dataloader"
        file = self.files[index]

        field_names = self.field_names
        fields = {}
        coords = []
        with h5py.File(file, mode="r") as h5file:
            data_file = h5file["tasks"]
            keys = list(data_file.keys())
            if field_names is None:
                field_names = keys
            for field_name in field_names:
                if field_name in data_file:
                    field = data_file[field_name][:]
                    fields[field_name] = field
                else:
                    print(f"field name {field_name} not found")
        dataset = fields
        return dataset

    def get_coords(self, index):
        "Gets coordinates of t, x, y for dataloader"
        file = self.files[index]
        with h5py.File(file, mode="r") as h5file:
            data_file = h5file["tasks"]
            keys = list(data_file.keys())
            dims = data_file[keys[0]].dims

            ndims = len(dims)
            t = dims[0]["sim_time"][:]
            x = dims[ndims - 2][0][:]
            y = dims[ndims - 1][0][:]
        return t, x, y
```

And the dataloader which is sampled from during training.

```python
class MHDDataloaderVecPot(Dataset):
    "Dataloader for MHD Dataset with vector potential"

    def __init__(
        self, dataset: Dedalus2DDataset, sub_x=1, sub_t=1, ind_x=None, ind_t=None
    ):
        self.dataset = dataset
        self.sub_x = sub_x
        self.sub_t = sub_t
        self.ind_x = ind_x
        self.ind_t = ind_t
        t, x, y = dataset.get_coords(0)
        self.x = x[:ind_x:sub_x]
        self.y = y[:ind_x:sub_x]
        self.t = t[:ind_t:sub_t]
        self.nx = len(self.x)
        self.ny = len(self.y)
        self.nt = len(self.t)
        self.num = num = len(self.dataset)
        self.x_slice = slice(0, self.ind_x, self.sub_x)
        self.t_slice = slice(0, self.ind_t, self.sub_t)

    def __len__(self):
        length = len(self.dataset)
        return length

    def __getitem__(self, index):
        "Gets input of dataloader, including data, t, x, and y"
        fields = self.dataset[index]

        # Data includes velocity and vector potential
        velocity = fields["velocity"]
        vector_potential = fields["vector potential"]

        u = torch.from_numpy(
            velocity[
                : self.ind_t : self.sub_t,
                0,
                : self.ind_x : self.sub_x,
                : self.ind_x : self.sub_x,
            ]
        )
        v = torch.from_numpy(
            velocity[
                : self.ind_t : self.sub_t,
                1,
                : self.ind_x : self.sub_x,
                : self.ind_x : self.sub_x,
            ]
        )
        A = torch.from_numpy(
            vector_potential[
                : self.ind_t : self.sub_t,
                : self.ind_x : self.sub_x,
                : self.ind_x : self.sub_x,
            ]
        )

        # shape is now (self.nt, self.nx, self.ny, nfields)
        data = torch.stack([u, v, A], dim=-1)
        data0 = data[0].reshape(1, self.nx, self.ny, -1).repeat(self.nt, 1, 1, 1)

        grid_t = (
            torch.from_numpy(self.t)
            .reshape(self.nt, 1, 1, 1)
            .repeat(1, self.nx, self.ny, 1)
        )
        grid_x = (
            torch.from_numpy(self.x)
            .reshape(1, self.nx, 1, 1)
            .repeat(self.nt, 1, self.ny, 1)
        )
        grid_y = (
            torch.from_numpy(self.y)
            .reshape(1, 1, self.ny, 1)
            .repeat(self.nt, self.nx, 1, 1)
        )

        inputs = torch.cat([grid_t, grid_x, grid_y, data0], dim=-1)
        outputs = data

        return inputs, outputs
    
    def create_dataloader(
        self,
        batch_size=1,
        shuffle=False,
        num_workers=0,
        pin_memory=False,
        distributed=False,
    ):
        "Creates dataloader and sampler based on whether distributed training is on"
        if distributed:
            sampler = torch.utils.data.DistributedSampler(self)
            dataloader = DataLoader(
                self,
                batch_size=batch_size,
                shuffle=False,
                sampler=sampler,
                num_workers=num_workers,
                pin_memory=pin_memory,
            )
        else:
            sampler = None
            dataloader = DataLoader(
                self,
                batch_size=batch_size,
                shuffle=shuffle,
                num_workers=num_workers,
                pin_memory=pin_memory,
            )

        return dataloader, sampler
```

## Model Architecture
![model_arch](model_arch.png)

Our PINO model is composed of Tensor Factorized Neural Operators as the core component. Input fields are fed in as the input, which are composed of $u$, $v$, and $A$ initial conditions. The data is first lifted into a higher dimension representation by the neural network, P1. The data then enters the Fourier layers ($F_1$,...,$F_n$). Each Fourier layer consists of a sequence of non-logical integral operators, and nonlinear activation functions. $T_1$ represents a linear transform that employs CP decomposed tensors as weights, and $T_2$ represents a local linear transform. $\sigma$ is the activation function, and $\mathcal{F}$, $\mathcal{F}^{-1}$ represent the Fourier transfrom and inverse Fourier transform respectively. At the end, $P_2$ projects back down into the input space, producing the output shown on the right which describes the next state of the system. This process is repeated to predict the time evolution of the system. 

## Training our Model

PhysicsNeMo has two distinct styles, namely Core and Sym. PhysicsNeMo Sym is a framework providing pythonic APIs, algorithms and utilities to be used with PhysicsNeMo Core, while PhysicsNeMo Core interoperates with PyTorch directly. Working with PhysicsNeMo Core looks and feels more like a PyTorch workflow with some key utils like models, utils, and datapipes imported directly from `physicsnemo` itself. While some components of this workflow so far have borrowed from PhysicsNeMo Sym (`MHD_PDE`), the training workflow for this problem will be build primarily using the Core style. This will provide more flexibility over our training loop, and allow for further customizations to our workflow. The training script follows the standard flow of training models using pytorch. 


## Hydra Config

Training in PhysicsNeMo is facilitated by Hydra configs, which allow us to set and manager parameters from a single file, updating parameters for components such as our model, datasets, optimizer, logger, loss function, and dataloaders. The first step in getting set up for training is defining this yaml file and loading the config.


```yaml 

## Training options
load_ckpt: False
use_wandb: True
output_dir: './checkpoints/MHD/MHD_PINO_Re250/figures/'
derivative: 'physicsnemo' # use 'physicsnemo' or 'original' method to take derivatives

###################
## Model options
model_params:
  layers: 64
  modes: 8
  num_fno_layers: 4
  fc_dim: 128
  decoder_layers: 1
  in_dim: 7 # 3 + in_fields
  out_dim: 4
  dimension: 3
  activation: 'gelu'
  pad_x: 5
  pad_y: 0
  pad_z: 0
  input_norm: [1.0, 1.0, 1.0, 1.0, 1.0, 0.01, 0.01]
  output_norm: [1.0, 1.0, 0.01, 0.01]

###################
## Dataset options
dataset_params:
  data_dir: 'mhd_data/simulation_outputs_Re250'
  field_names: ['velocity', 'magnetic field']
  output_names: 'output-????'
  dataset_type: 'mhd'
  name: 'MHD_Re250'
  num: 100
  num_train: 90
  num_test: 10
  sub_x: 1
  sub_t: 2
  ind_x: null
  ind_t: 51
  nin: 4
  nout: 4
  fields: ['u', 'v', 'Bx', By]

###################
## Dataloader options
train_loader_params:
  batch_size: 1
  shuffle: True
  num_workers: 4
  pin_memory: True

val_loader_params:
  batch_size: 1
  shuffle: False
  num_workers: 4
  pin_memory: True

test_loader_params:
  batch_size: 1
  shuffle: False
  num_workers: 4
  pin_memory: True

###################
## Loss options
loss_params:
  nu: 0.004
  eta: 0.004
  rho0: 1.0
  
  data_weight: 5.0 # 2.0
  ic_weight: 1.0
  pde_weight: 1.0
  constraint_weight: 1.0
  
  use_data_loss: True
  use_ic_loss: True
  use_pde_loss: True
  use_constraint_loss: True
  
  u_weight: 1.0
  v_weight: 1.0
  Bx_weight: 1.0
  By_weight: 1.0
  
  Du_weight: 1.0
  Dv_weight: 1.0
  DBx_weight: 1_000 #1.0
  DBy_weight: 1_000 #1.0
  
  div_B_weight: 1.0
  div_vel_weight: 1.0
  
  Lx: 1.0
  Ly: 1.0
  tend: 0.5 #1.0

  use_weighted_mean: False
  
###################
## Optimizer options
optimizer_params:
  betas: [0.9, 0.999]
  lr: 0.001
  milestones: [25, 50, 75, 100, 125, 150]
  gamma: 0.5
  weight_decay: 0.1


###################
## Train params
train_params:
  epochs: 150
  ckpt_freq: 25
  ckpt_path: 'checkpoints/MHD/MHD_PINO_Re250/'

###################
## wandb params
wandb_params:
  wandb_dir: 'logs'
  wandb_project: 'MHD_PINO'
  wandb_group: 'MHD_Re250'
  wandb_num_plots: 1
  wandb_plot_freq: 5
  wandb_plot_types: ['ic', 'pred', 'true', 'error']

test:
  batchsize: 1
  ckpt_path: 'checkpoints/MHD/MHD_PINO_Re250/'

```

## Training Setup

We begin with importing the required modules, capturing our hydra config, and initializing some utilities to facilitate the model training. Most of this initial setup is 

```python
import os

import hydra
import plotly
import torch
import wandb
from omegaconf import DictConfig
from physicsnemo.distributed import DistributedManager
from physicsnemo.launch.logging import LaunchLogger, PythonLogger
from physicsnemo.launch.logging.wandb import initialize_wandb
from physicsnemo.launch.utils import load_checkpoint, save_checkpoint
from physicsnemo.sym.hydra import to_absolute_path
from torch.nn.parallel import DistributedDataParallel
from torch.optim import AdamW

from dataloaders import Dedalus2DDataset, MHDDataloaderVecPot
from losses import LossMHDVecPot_PhysicsNeMo
from tfno import TFNO
from utils.plot_utils import plot_predictions_mhd, plot_predictions_mhd_plotly

dtype = torch.float # dtype = torch.double
torch.set_default_dtype(dtype)


@hydra.main(
    version_base="1.3", config_path="config", config_name="mhd_vec_pot_tfno_Re250.yaml"
)
def main(cfg: DictConfig) -> None:
    DistributedManager.initialize()  # Only call this once in the entire script!
    dist = DistributedManager()  # call if required elsewhere

    # initialize monitoring
    log = PythonLogger(name="mhd_pino")
    log.file_logging()

    wandb_dir = cfg.wandb_params.wandb_dir
    wandb_project = cfg.wandb_params.wandb_project
    wandb_group = cfg.wandb_params.wandb_group

    initialize_wandb(
        project=wandb_project,
        entity="pino_trainer",
        mode="offline",
        group=wandb_group,
        config=dict(cfg),
        results_dir=wandb_dir,
    )

    LaunchLogger.initialize(use_wandb=cfg.use_wandb)  # PhysicsNeMo launch logger

    # Load config file parameters
    model_params = cfg.model_params
    dataset_params = cfg.dataset_params
    train_loader_params = cfg.train_loader_params
    val_loader_params = cfg.val_loader_params
    test_loader_params = cfg.test_loader_params
    loss_params = cfg.loss_params
    optimizer_params = cfg.optimizer_params
    train_params = cfg.train_params
    wandb_params = cfg.wandb_params

    load_ckpt = cfg.load_ckpt
    output_dir = cfg.output_dir
    use_wandb = cfg.use_wandb

    output_dir = to_absolute_path(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    data_dir = dataset_params.data_dir
    ckpt_path = train_params.ckpt_path
    wandb_dir = wandb_params.wandb_dir
    wandb_project = wandb_params.wandb_group
    wandb_group = wandb_params.wandb_project
```

## Datasets and Dataloaders

Datasets and dataloaders are initialized using parameters from the hydra config.

```python
# Construct dataloaders
dataset_train = Dedalus2DDataset(
    dataset_params.data_dir,
    output_names=dataset_params.output_names,
    field_names=dataset_params.field_names,
    num_train=dataset_params.num_train,
    num_test=dataset_params.num_test,
    use_train=True,
)
dataset_val = Dedalus2DDataset(
    data_dir,
    output_names=dataset_params.output_names,
    field_names=dataset_params.field_names,
    num_train=dataset_params.num_train,
    num_test=dataset_params.num_test,
    use_train=False,
)

mhd_dataloader_train = MHDDataloaderVecPot(
    dataset_train,
    sub_x=dataset_params.sub_x,
    sub_t=dataset_params.sub_t,
    ind_x=dataset_params.ind_x,
    ind_t=dataset_params.ind_t,
)
mhd_dataloader_val = MHDDataloaderVecPot(
    dataset_val,
    sub_x=dataset_params.sub_x,
    sub_t=dataset_params.sub_t,
    ind_x=dataset_params.ind_x,
    ind_t=dataset_params.ind_t,
)

dataloader_train, sampler_train = mhd_dataloader_train.create_dataloader(
    batch_size=train_loader_params.batch_size,
    shuffle=train_loader_params.shuffle,
    num_workers=train_loader_params.num_workers,
    pin_memory=train_loader_params.pin_memory,
    distributed=dist.distributed,
)
dataloader_val, sampler_val = mhd_dataloader_val.create_dataloader(
    batch_size=val_loader_params.batch_size,
    shuffle=val_loader_params.shuffle,
    num_workers=val_loader_params.num_workers,
    pin_memory=val_loader_params.pin_memory,
    distributed=dist.distributed,
)
```

## Model Construction
For a relatively simple model such as `FNO`, we can directly use an architecture pre-defined by PhysicsNeMo. Hyper-parameters are set directly from the hydra config, and makes it straight forward to configure hyper parameter optimization if necessary. For a more complex model such as `tFNO`, we can leverage a combination of PhysicsNeMo primitives and third party packages to build a model in pytorch. 

```python
# define FNO model
model = TFNO(
    in_channels=model_params.in_dim,
    out_channels=model_params.out_dim,
    decoder_layers=model_params.decoder_layers,
    decoder_layer_size=model_params.fc_dim,
    dimension=model_params.dimension,
    latent_channels=model_params.layers,
    num_fno_layers=model_params.num_fno_layers,
    num_fno_modes=model_params.modes,
    padding=[model_params.pad_z, model_params.pad_y, model_params.pad_x],
    rank=model_params.rank,
    factorization=model_params.factorization,
    fixed_rank_modes=model_params.fixed_rank_modes,
    decomposition_kwargs=model_params.decomposition_kwargs,
).to(dist.device)

# Set up DistributedDataParallel if using more than a single process.
# The `distributed` property of DistributedManager can be used to
# check this.
if dist.distributed:
    ddps = torch.cuda.Stream()
    with torch.cuda.stream(ddps):
        model = DistributedDataParallel(
            model,
            device_ids=[dist.local_rank],  # Set the device_id to be
            # the local rank of this process on
            # this node
            output_device=dist.device,
            broadcast_buffers=dist.broadcast_buffers,
            find_unused_parameters=dist.find_unused_parameters,
        )
    torch.cuda.current_stream().wait_stream(ddps)
```

## Optimizer, Scheduler, Loss Functions and Check-pointing


```python
# Construct optimizer and scheduler
optimizer = AdamW(
    model.parameters(),
    betas=optimizer_params.betas,
    lr=optimizer_params.lr,
    weight_decay=0.1,
)

scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=optimizer_params.milestones, gamma=optimizer_params.gamma
)

# Construct Loss class
mhd_loss = LossMHDVecPot_PhysicsNeMo(**loss_params)

# Load model from checkpoint (if exists)
loaded_epoch = 0
if load_ckpt:
    loaded_epoch = load_checkpoint(
        ckpt_path, model, optimizer, scheduler, device=dist.device
    )
```

## Training Loop
Finally, the main training loop iterates through the dataset for our defined number of epochs, saving checkpoints and visualizations of our training along the way.

```python
# Training Loop
epochs = train_params.epochs
ckpt_freq = train_params.ckpt_freq
names = dataset_params.fields
input_norm = torch.tensor(model_params.input_norm).to(dist.device)
output_norm = torch.tensor(model_params.output_norm).to(dist.device)

for epoch in range(max(1, loaded_epoch + 1), epochs + 1):
    with LaunchLogger(
        "train",
        epoch=epoch,
        num_mini_batch=len(dataloader_train),
        epoch_alert_freq=1,
    ) as log:

        if dist.distributed:
            sampler_train.set_epoch(epoch)

        # Train Loop
        model.train()

        for i, (inputs, outputs) in enumerate(dataloader_train):
            inputs = inputs.type(torch.FloatTensor).to(dist.device)
            outputs = outputs.type(torch.FloatTensor).to(dist.device)
            # Zero Gradients
            optimizer.zero_grad()
            # Compute Predictions
            pred = (
                model((inputs / input_norm).permute(0, 4, 1, 2, 3)).permute(
                    0, 2, 3, 4, 1
                )
                * output_norm
            )
            # Compute Loss
            loss, loss_dict = mhd_loss(pred, outputs, inputs, return_loss_dict=True)
            # Compute Gradients for Back Propagation
            loss.backward()
            # Update Weights
            optimizer.step()

            log.log_minibatch(loss_dict)

        log.log_epoch({"Learning Rate": optimizer.param_groups[0]["lr"]})
        scheduler.step()

    with LaunchLogger("valid", epoch=epoch) as log:
        # Val loop
        model.eval()
        val_loss_dict = {}
        plot_count = 0
        plot_dict = {name: {} for name in names}
        with torch.no_grad():
            for i, (inputs, outputs) in enumerate(dataloader_val):
                inputs = inputs.type(dtype).to(dist.device)
                outputs = outputs.type(dtype).to(dist.device)

                # Compute Predictions
                pred = (
                    model((inputs / input_norm).permute(0, 4, 1, 2, 3)).permute(
                        0, 2, 3, 4, 1
                    )
                    * output_norm
                )
                # Compute Loss
                loss, loss_dict = mhd_loss(
                    pred, outputs, inputs, return_loss_dict=True
                )

                log.log_minibatch(loss_dict)

                # Get prediction plots to log for wandb
                # Do for number of batches specified in the config file
                if (i < wandb_params.wandb_num_plots) and (
                    epoch % wandb_params.wandb_plot_freq == 0
                ):
                    # Add all predictions in batch
                    for j, _ in enumerate(pred):
                        # Make plots for each field
                        for index, name in enumerate(names):
                            # Generate figure
                            if use_wandb:
                                figs = plot_predictions_mhd_plotly(
                                    pred[j].cpu(),
                                    outputs[j].cpu(),
                                    inputs[j].cpu(),
                                    index=index,
                                    name=name,
                                )
                                # Add figure to plot dict
                                plot_dict[name] = {
                                    f"{plot_type}-{plot_count}": wandb.Html(
                                        plotly.io.to_html(fig)
                                    )
                                    for plot_type, fig in zip(
                                        wandb_params.wandb_plot_types, figs
                                    )
                                }

                        plot_count += 1

                # Get prediction plots and save images locally
                if (i < 2) and (epoch % wandb_params.wandb_plot_freq == 0):
                    # Add all predictions in batch
                    for j, _ in enumerate(pred):
                        # Generate figure
                        plot_predictions_mhd(
                            pred[j].cpu(),
                            outputs[j].cpu(),
                            inputs[j].cpu(),
                            names=names,
                            save_path=os.path.join(
                                output_dir,
                                "MHD_" + cfg.derivative + "_" + str(dist.rank),
                            ),
                            save_suffix=i,
                        )

        if use_wandb and epoch % wandb_params["wandb_plot_freq"] == 0:
            wandb.log({"plots": plot_dict})

        if epoch % ckpt_freq == 0 and dist.rank == 0:
            save_checkpoint(ckpt_path, model, optimizer, scheduler, epoch=epoch)


```

## Evaluation
![ModelOutput](MHD_0_0.png)

## Transfer Learning to New Reynolds Number
In practice, our system may not follow smooth, laminar flows described with low Reynolds numbers. In MHD systems, much of the magnetic field energy is stored at high wave numbers, which occur at smaller scales. Models must then be able to characterize high frequency features in order to successfully reproduce the trajectories of the system. These turbulent flows at higher Reynolds number are simulated, which will in turn produce higher frequency features that a model trained on smooth flows may not be able to resolve with good accuracy. To this end, transfer learning can be used to take a base model and adapt it to the new data domain by using a pre-trained checkpoint as the starting point of a new iteration of model training. 

![high_frequency](high_frequency.png)

## Shortcomings and areas for improvement